# A Part 2
5. Write a query to define and calculate the RFM values per customer.
6. Check the distribution of Recency, Frequency and Monetary Values.
7. Briefly discuss the issue of skewness and remove skew from the data.


# A.5
1. Calculate Recency based on last transaction date + 1 day.
2. Calculate Frequency based on CustomerID count.
3. Calculate Monerary based on total transaction value.
4. Concatenate into RFM Model.


In [ ]:
import pandas

# Read CSV and convert transaction date from string to datetime for analysis

df = pandas.read_csv('bank_transactions_cleaned.csv')
df['TransactionDate'] = pandas.to_datetime(df['TransactionDate'])
# reference date is the last transaction date in the data frame + 1 day
referenceDate = df['TransactionDate'].max() + pandas.Timedelta(days=1)


<ipython-input-1-061760fbc68a>:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['TransactionDate'] = pandas.to_datetime(df['TransactionDate'])


In [ ]:
# 1.
# get the last purchase date of the customer
# subtract this from the last transaction date in the df + 1 (the reference date)
# THE REASON WE USE THE LAST DATE + 1 IS SO THAT EVEN IF THE DATA SET IS FROM YEARS AGO YOU CAN-
# CALCULATE RECENCY RELATIVE TO THE DATA SET AND NOT TO TODAY'S DATE.

def calculateRec(df, refDate) :
    lastPurchaseDate = df.groupby('CustomerID')['TransactionDate'].max()
    recency = refDate - lastPurchaseDate
    recencyInDays = recency.dt.days
    return recencyInDays

#calculateRec(df, referenceDate)

In [ ]:
# 2.
# return the count for each CustomerID


def calculateFreq(df) :
  return df.groupby('CustomerID')['TransactionID'].nunique()


#calculateFreq(df)
print(df['CustomerID'].nunique()) # returns number of unique customer ids
print(df.groupby('CustomerID')['TransactionID'].count().value_counts()) # returns number of transaction ids grouped by customer id (how many transactions per customer)

884265
TransactionID
1    740653
2    124960
3     16789
4      1702
5       147
6        14
Name: count, dtype: int64


In [ ]:
# 3.
# return the total transaction value per customer

def calculateMon(df) :
  return df.groupby('CustomerID')['TransactionAmount (INR)'].max()

#calculateMon(df)

In [ ]:
# 4.
# concatenate data into RFM model

rfm = pandas.DataFrame({
    'Recency': calculateRec(df, referenceDate),
    'Frequency': calculateFreq(df),
    'Monetary': calculateMon(df)
})

print(rfm.head())

sortByRecency = rfm.sort_values('Recency', ascending=False)
sortByFrequency = rfm.sort_values('Frequency', ascending=False)
sortByMonetary = rfm.sort_values('Monetary', ascending=False)
print(sortByRecency.head())
print(sortByFrequency.head())
print(sortByMonetary.head())

            Recency  Frequency  Monetary
CustomerID                              
C1010011         75          2    4750.0
C1010012        118          1    1499.0
C1010014        155          2    1205.0
C1010018         86          1      30.0
C1010024        114          1    5000.0
            Recency  Frequency  Monetary
CustomerID                              
C8437020        337          1     440.0
C1010078        337          1     124.0
C8436992        337          1    2800.0
C8437128        337          1    5952.0
C5237264        337          1    4970.0
            Recency  Frequency  Monetary
CustomerID                              
C1026833        101          6   1245.00
C5531319         31          6   5000.00
C1113684        104          6  34980.00
C3226689         71          6   6351.39
C6735477          1          6   2000.00
            Recency  Frequency    Monetary
CustomerID                                
C7319271        114          1  1560034.99
C6677159  

# A.5 cont.
assign percentage value to rfm

In [ ]:
# Percentage Values for Recency

def RecencyPerc(df) :
  perc = (df / df.max()) * 100
  return 100 - perc

print(RecencyPerc(rfm['Recency']).round())

CustomerID
C1010011    78.0
C1010012    65.0
C1010014    54.0
C1010018    74.0
C1010024    66.0
            ... 
C9099836    54.0
C9099877    74.0
C9099919    66.0
C9099941    69.0
C9099956    77.0
Name: Recency, Length: 884265, dtype: float64


In [ ]:
# Percentage Values for Frequency or Monetary

def FrequencyOrMonetaryPerc(df) :
  return (df / df.max()) * 100

In [ ]:
test = pandas.DataFrame({
    'Recency': RecencyPerc(rfm['Recency']),
    'Frequency': FrequencyOrMonetaryPerc(rfm['Frequency']),
    'Monetary': FrequencyOrMonetaryPerc(rfm['Monetary'])
})

print(test.round())

            Recency  Frequency  Monetary
CustomerID                              
C1010011       78.0       33.0       0.0
C1010012       65.0       17.0       0.0
C1010014       54.0       33.0       0.0
C1010018       74.0       17.0       0.0
C1010024       66.0       17.0       0.0
...             ...        ...       ...
C9099836       54.0       17.0       0.0
C9099877       74.0       17.0       0.0
C9099919       66.0       17.0       0.0
C9099941       69.0       17.0       0.0
C9099956       77.0       17.0       0.0

[884265 rows x 3 columns]


In [ ]:
print(test[test['Recency'] >= 80])

              Recency  Frequency  Monetary
CustomerID                                
C1010041    81.602374  50.000000  0.929466
C1010068    81.305638  16.666667  0.034999
C1010072    90.801187  16.666667  0.224995
C1010087    90.504451  16.666667  0.002692
C1010116    99.406528  16.666667  0.002564
...               ...        ...       ...
C9097954    90.504451  16.666667  0.012820
C9098073    81.602374  16.666667  0.031986
C9098516    81.305638  33.333333  0.243584
C9099116    81.305638  16.666667  0.003205
C9099692    85.163205  16.666667  0.015961

[138211 rows x 3 columns]


In [ ]:
# assign customer values based on results

def CustomerAssignment(row):
  if row['Recency'] >= 80 & row['Frequency'] >= 80 & row['Monetary'] >= 80:
    return 'Best Customer'
  elif row['Recency'] >= 60 & row['Frequency'] >= 60 & row['Monetary'] >= 60:
    return 'Loyal'
  elif row['Recency'] >= 60 & row['Frequency'] >= 20 & row['Monetary'] >= 40:
    return 'Potential Loyalist'
  elif row['Recency'] >= 60 & row['Frequency'] >= 20 & row['Monetary'] >= 20:
    return 'Promising'
  elif row['Recency'] >= 40 & row['Frequency'] >= 40 & row['Monetary'] >= 40:
    return 'Customers Needing Attention'
  elif row['Recency'] >= 20 & row['Frequency'] >= 40 & row['Monetary'] >= 40:
    return 'At Risk'
  elif row['Recency'] >= 20 & row['Frequency'] >= 20 & row['Monetary'] >= 40:
    return 'Hibernating'
  else:
    return 'Lost'

test[['Recency', 'Frequency', 'Monetary']] = test[['Recency', 'Frequency', 'Monetary']].astype(int)

test['Type'] = test.apply(CustomerAssignment, axis=1)
print(test)



            Recency  Frequency  Monetary  Type
CustomerID                                    
C1010011         77         33         0  Lost
C1010012         64         16         0  Lost
C1010014         54         33         0  Lost
C1010018         74         16         0  Lost
C1010024         66         16         0  Lost
...             ...        ...       ...   ...
C9099836         54         16         0  Lost
C9099877         74         16         0  Lost
C9099919         66         16         0  Lost
C9099941         69         16         0  Lost
C9099956         77         16         0  Lost

[884265 rows x 4 columns]


In [ ]:
# For A.5.4 (creating rfm model) testing we discovered that only one customer fell into a category outside of 'Lost'
# this was customer C7525532 who was Promising. This is because the RFM definitions (boundaries) for Customer Types did not take into account
# the skewness of the high monetary purchases which meant that only a customer making massive transactions regularly and recently
# would class as a 'Best Customer' which is unrealistic.
#
# This means the Customer Type boundaries need to be changed to better gather information from this data model.

print(test[test['Type'] == 'Best Customer'])
print(test[test['Type'] == 'Loyal'])
print(test[test['Type'] == 'Potential Loyalist'])
print(test[test['Type'] == 'Promising'])
print(test[test['Type'] == 'Customers Needing Attention'])
print(test[test['Type'] == 'At Risk'])
print(test[test['Type'] == 'Hibernating'])
print(test[test['Type'] == 'Lost'])

Empty DataFrame
Columns: [Recency, Frequency, Monetary, Type]
Index: []
Empty DataFrame
Columns: [Recency, Frequency, Monetary, Type]
Index: []
Empty DataFrame
Columns: [Recency, Frequency, Monetary, Type]
Index: []
            Recency  Frequency  Monetary       Type
CustomerID                                         
C3912568         67         33        22  Promising
C7525532         65         50        28  Promising
Empty DataFrame
Columns: [Recency, Frequency, Monetary, Type]
Index: []
Empty DataFrame
Columns: [Recency, Frequency, Monetary, Type]
Index: []
Empty DataFrame
Columns: [Recency, Frequency, Monetary, Type]
Index: []
            Recency  Frequency  Monetary  Type
CustomerID                                    
C1010011         77         33         0  Lost
C1010012         64         16         0  Lost
C1010014         54         33         0  Lost
C1010018         74         16         0  Lost
C1010024         66         16         0  Lost
...             ...        ... 

In [ ]:
# it turns out there are only two transactions in the top 20 percentile.

print(test[test['Monetary'] >= 80])

            Recency  Frequency  Monetary  Type
CustomerID                                    
C6677159         67         16        88  Lost
C7319271         66         16       100  Lost


In [ ]:
# assign customers with new rfm boundaries

def CustomerAssignment(row):
  if row['Recency'] >= 0 & row['Frequency'] >= 0 & row['Monetary'] >= 80:
    return 'High Paying Customer'
  elif row['Recency'] >= 80 & row['Frequency'] >= 80 & row['Monetary'] < 80:
    return 'Best Customer'
  elif row['Recency'] >= 80 & row['Frequency'] >= 60 & row['Monetary'] >= 0:
    return 'Loyal'
  elif row['Recency'] >= 60 & row['Frequency'] >= 60 & row['Monetary'] >= 0:
    return 'Potential Loyalist'
  elif row['Recency'] >= 60 & row['Frequency'] >= 40 & row['Monetary'] >= 0:
    return 'Promising'
  elif row['Recency'] >= 40 & row['Frequency'] >= 40 & row['Monetary'] >= 0:
    return 'Customers Needing Attention'
  elif row['Recency'] >= 40 & row['Frequency'] >= 20 & row['Monetary'] >= 0:
    return 'At Risk'
  elif row['Recency'] >= 20 & row['Frequency'] >= 20 & row['Monetary'] >= 0:
    return 'Hibernating'
  else:
    return 'Lost'

test[['Recency', 'Frequency', 'Monetary']] = test[['Recency', 'Frequency', 'Monetary']].astype(int)

test['new_type'] = test.apply(CustomerAssignment, axis=1)
print(test)

            Recency  Frequency  Monetary  Type       new_type
CustomerID                                                   
C1010011         77         33         0  Lost  Best Customer
C1010012         64         16         0  Lost  Best Customer
C1010014         54         33         0  Lost  Best Customer
C1010018         74         16         0  Lost  Best Customer
C1010024         66         16         0  Lost  Best Customer
...             ...        ...       ...   ...            ...
C9099836         54         16         0  Lost  Best Customer
C9099877         74         16         0  Lost  Best Customer
C9099919         66         16         0  Lost  Best Customer
C9099941         69         16         0  Lost  Best Customer
C9099956         77         16         0  Lost  Best Customer

[884265 rows x 5 columns]


In [ ]:
print(test[test['new_type'] == 'Best Customer'])
print(test[test['new_type'] == 'Loyal'])
print(test[test['new_type'] == 'Potential Loyalist'])
print(test[test['new_type'] == 'Promising'])
print(test[test['new_type'] == 'Customers Needing Attention'])
print(test[test['new_type'] == 'At Risk'])
print(test[test['new_type'] == 'Hibernating'])
print(test[test['new_type'] == 'Lost'])
print(test[test['new_type'] == 'High Paying Customer'])

            Recency  Frequency  Monetary  Type       new_type
CustomerID                                                   
C1010011         77         33         0  Lost  Best Customer
C1010012         64         16         0  Lost  Best Customer
C1010014         54         33         0  Lost  Best Customer
C1010018         74         16         0  Lost  Best Customer
C1010024         66         16         0  Lost  Best Customer
...             ...        ...       ...   ...            ...
C9099836         54         16         0  Lost  Best Customer
C9099877         74         16         0  Lost  Best Customer
C9099919         66         16         0  Lost  Best Customer
C9099941         69         16         0  Lost  Best Customer
C9099956         77         16         0  Lost  Best Customer

[823084 rows x 5 columns]
Empty DataFrame
Columns: [Recency, Frequency, Monetary, Type, new_type]
Index: []
            Recency  Frequency  Monetary  Type            new_type
CustomerID        